In [1]:
import requests

In [4]:
apiEndPoints = {
    "album": r"https://api.deezer.com/album/{0}",
    "artist": r"https://api.deezer.com/artist/{0}",
    "track": r"https://api.deezer.com/track/{0}"    
}

In [16]:
access_token = "frBx6ZdRjE553f9dc2a3cc9P7JptufC553f9dc2a3d00hX24wk"

In [12]:
def get_item_from_api(itemType, itemId):
    r = requests.get(url = apiEndPoints[itemType].format(itemId))
    return r.json()

In [13]:
get_item_from_api("artist", 27)

{'id': 27,
 'link': 'https://www.deezer.com/artist/27',
 'name': 'Daft Punk',
 'nb_album': 30,
 'nb_fan': 3025968,
 'picture': 'https://api.deezer.com/artist/27/image',
 'picture_big': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/500x500-000000-80-0-0.jpg',
 'picture_medium': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/250x250-000000-80-0-0.jpg',
 'picture_small': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/56x56-000000-80-0-0.jpg',
 'radio': True,
 'share': 'http://www.deezer.com/artist/27?utm_source=deezer&utm_content=artist-27&utm_term=0_1454754910&utm_medium=web',
 'tracklist': 'https://api.deezer.com/artist/27/top?limit=50',
 'type': 'artist'}

In [19]:
def get_history():
    url = r"http://api.deezer.com/user/me/history?access_token=" + access_token
    return requests.get(url = url).json()

In [20]:
get_history()

MissingSchema: Invalid URL 'api.deezer.com/user/me/history?access_token=frBx6ZdRjE553f9dc2a3cc9P7JptufC553f9dc2a3d00hX24wk': No schema supplied. Perhaps you meant http://api.deezer.com/user/me/history?access_token=frBx6ZdRjE553f9dc2a3cc9P7JptufC553f9dc2a3d00hX24wk?